# 4. Automatic differentiation

In [1]:
import tensorflow as tf
import numpy as np

2024-03-11 10:26:46.463612: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 10:26:46.542356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 10:26:46.542392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 10:26:46.544552: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 10:26:46.558909: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 10:26:46.559857: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
x=tf.constant([1,2], dtype=tf.int32)
with tf.GradientTape() as tape:
    y = x**2
print(tape.gradient(y,x))

None


In [11]:
x=tf.constant([1,2])
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x**2
print(tape.gradient(y,x))

None


In [12]:
x=tf.constant([1,2], dtype=tf.float16)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x**2
print(tape.gradient(y,x))

tf.Tensor([2. 4.], shape=(2,), dtype=float16)


In [13]:
x=tf.Variable([1,2], dtype=tf.float16)
with tf.GradientTape() as tape:
    # tape.watch(x)
    y = x**2
print(tape.gradient(y,x))

tf.Tensor([2. 4.], shape=(2,), dtype=float16)


In [20]:
w=tf.Variable(tf.random.normal(shape=(3,2)),name="w")
b=tf.Variable(tf.random.normal(shape=(1,2)),name="b")
x=[[1.,2.,3.]]

with tf.GradientTape() as tape:
    y = x@w+b
    loss=tf.reduce_mean(y**2)
    print("loss : ", loss)
tape.gradient(loss, (w,b))

loss :  tf.Tensor(13.7238655, shape=(), dtype=float32)


(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 5.218515  ,  0.46350056],
        [10.43703   ,  0.9270011 ],
        [15.655544  ,  1.3905017 ]], dtype=float32)>,
 <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[5.218515  , 0.46350056]], dtype=float32)>)

In [26]:
layer = tf.keras.layers.Dense(units=2, activation='relu')

x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
    y = layer(x)
    loss = tf.reduce_mean(y**2)

tape.gradient(loss, layer.trainable_weights)

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[2.3630042, 0.       ],
        [4.7260084, 0.       ],
        [7.0890126, 0.       ]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2.3630042, 0.       ], dtype=float32)>]

In [27]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
# Not a variable
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g)

tf.Tensor(6.0, shape=(), dtype=float32)
None
None
None


In [39]:
x0 = tf.Variable(1.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x1)
  tape.watch(x0)
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

grad = tape.gradient(ys, {'x0': x0, 'x1': x1})
grad

{'x0': <tf.Tensor: shape=(), dtype=float32, numpy=0.54030234>,
 'x1': <tf.Tensor: shape=(), dtype=float32, numpy=0.9999546>}

In [ ]:
x = tf.constant([1, 3.0])
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  y = x * x
  z = y * y

print(tape.gradient(z, x).numpy())  # [4.0, 108.0] (4 * x**3 at x = [1.0, 3.0])
print(tape.gradient(y, x).numpy())  # [2.0, 6.0] (2 * x at x = [1.0, 3.0])